In [3]:
import blpapi
from datetime import datetime
import pandas as pd
import time

# Bloomberg API Session
def get_bloomberg_session():
    sessionOptions = blpapi.SessionOptions()
    sessionOptions.setServerHost("localhost")  # Bloomberg Terminal must be running
    sessionOptions.setServerPort(8194)  # Default port
    session = blpapi.Session(sessionOptions)

    if not session.start():
        print("Failed to start session.")
        return None

    return session

# Request Tick Data
def get_tick_data(security, start_date, end_date, event_types=["TRADE", "BID", "ASK"]):
    session = get_bloomberg_session()
    if session is None:
        return None

    refDataService = session.getService("//blp/refdata")
    request = refDataService.createRequest("IntradayTickRequest")

    # Set request parameters
    request.set("security", security)
    request.set("startDateTime", start_date.strftime("%Y-%m-%dT%H:%M:%S"))
    request.set("endDateTime", end_date.strftime("%Y-%m-%dT%H:%M:%S"))

    for event in event_types:
        request.append("eventTypes", event)

    session.sendRequest(request)

    # Process Response
    tick_data = []
    while True:
        event = session.nextEvent(500)
        for msg in event:
            if msg.messageType() == blpapi.Name("IntradayTickResponse"):
                for tick in msg.getElement("tickData").getElement("tickData"):
                    tick_data.append({
                        "time": tick.getElementAsDatetime("time"),
                        "type": tick.getElementAsString("type"),
                        "value": tick.getElementAsFloat("value"),
                        "size": tick.getElementAsInteger("size")
                    })
        if event.eventType() == blpapi.Event.RESPONSE:
            break

    return pd.DataFrame(tick_data)

# Example Usage
if __name__ == "__main__":
    security = "AAPL US Equity"  # Change to desired ticker
    start_date = datetime(2024, 1, 31, 9, 30)
    end_date = datetime(2024, 1, 31, 10, 00)

    df = get_tick_data(security, start_date, end_date)
    print(df)


InvalidArgumentException: Null service handle (0x00020002)

In [4]:
import blpapi
from datetime import datetime
import pandas as pd


# Bloomberg API session setup and tick data retrieval function
def get_tick_data(security, start_date, end_date, event_types=None):
    if event_types is None:
        event_types = ["TRADE"]

    # Step 1: Start a session
    session_options = blpapi.SessionOptions()
    session_options.setServerHost("localhost")  # Replace with actual host if needed
    session_options.setServerPort(8194)  # Default Bloomberg server port
    session = blpapi.Session(session_options)

    # Start the session and check if it started successfully
    if not session.start():
        print("Failed to start session.")
        return None

    # Step 2: Open the "//blp/refdata" service
    if not session.openService("//blp/refdata"):
        print("Failed to open service //blp/refdata.")
        return None

    # Access the service
    refDataService = session.getService("//blp/refdata")

    # Step 3: Create the request for Intraday Tick
    try:
        request = refDataService.createRequest("IntradayTickRequest")
    except blpapi.exception.InvalidArgumentException as e:
        print(f"Error creating request: {e}")
        return None

    # Step 4: Set request parameters
    request.set("security", security)
    request.set("startDateTime", start_date.strftime("%Y-%m-%dT%H:%M:%S"))
    request.set("endDateTime", end_date.strftime("%Y-%m-%dT%H:%M:%S"))

    # Add event types (default is TRADE)
    event_types_array = request.getElement("eventTypes")
    for event in event_types:
        event_types_array.appendValue(event)

    # Step 5: Send the request and process the response
    session.sendRequest(request)

    data = []

    while True:
        event = session.nextEvent()
        for msg in event:
            if msg.hasElement("tickData", True):  # Check if tickData is present
                tick_data = msg.getElement("tickData").getElement("tickData")
                for value in tick_data.values():
                    time = value.getElementAsDatetime("time")
                    type_ = value.getElementAsString("type")
                    value_field = value.getElement("value").getValueAsFloat() if value.hasElement("value") else None

                    data.append({"time": time, "type": type_, "value": value_field})

        # If event is final, break out of the loop
        if event.eventType() == blpapi.Event.RESPONSE:
            break

    # Step 6: Convert data to a DataFrame
    session.stop()

    if data:
        return pd.DataFrame(data)
    else:
        print("No data received.")
        return None


# Example usage
if __name__ == "__main__":
    security = "AAPL US Equity"
    start_date = datetime(2024, 1, 31, 9, 30)
    end_date = datetime(2024, 1, 31, 10, 0)

    df = get_tick_data(security, start_date, end_date)
    if df is not None:
        print(df)
    else:
        print("Failed to retrieve tick data.")

No data received.
Failed to retrieve tick data.


In [2]:
import blpapi
from datetime import datetime
import pandas as pd


# Bloomberg API Session
def get_bloomberg_session():
    session_options = blpapi.SessionOptions()
    session_options.setServerHost("localhost")  # Bloomberg Terminal must be running
    session_options.setServerPort(8194)  # Default port
    session = blpapi.Session(session_options)

    if not session.start():
        print("Failed to start session.")
        return None

    if not session.openService("//blp/refdata"):
        print("Failed to open service //blp/refdata.")
        return None

    return session


# Request Daily Close Price via Historical Data
def get_daily_close_price(security, start_date, end_date):
    session = get_bloomberg_session()
    if session is None:
        return None

    # Access the reference data service
    refDataService = session.getService("//blp/refdata")

    # Create Historical Data Request
    request = refDataService.createRequest("HistoricalDataRequest")

    # Set request parameters
    request.getElement("securities").appendValue(security)
    request.getElement("fields").appendValue("PX_LAST")  # Last (close) price
    request.set("startDate", start_date.strftime("%Y%m%d"))  # Bloomberg requires date in YYYYMMDD format
    request.set("endDate", end_date.strftime("%Y%m%d"))  # End date
    request.set("periodicitySelection", "DAILY")  # Set frequency to daily

    # Send the request
    session.sendRequest(request)

    # Collect response
    data = []
    while True:
        event = session.nextEvent(500)
        for msg in event:
            if msg.messageType() == blpapi.Name("HistoricalDataResponse"):
                security_data = msg.getElement("securityData")
                field_data = security_data.getElement("fieldData")

                # Extract date and close price
                for record in field_data.values():
                    data.append({
                        "date": record.getElementAsDatetime("date"),
                        "close": record.getElementAsFloat("PX_LAST")
                    })

        if event.eventType() == blpapi.Event.RESPONSE:
            break

    # Convert results into a DataFrame
    session.stop()
    return pd.DataFrame(data)


# Example Usage
if __name__ == "__main__":
    security = "AAPL US Equity"  # Change for different tickers
    start_date = datetime(2023, 1, 1)  # Start date
    end_date = datetime(2023, 12, 31)  # End date

    df = get_daily_close_price(security, start_date, end_date)
    if df is not None:
        print(df)
        df.to_csv("AAPL_daily_close.csv", index=False)  # Save to CSV

           date   close
0    2023-01-03  125.07
1    2023-01-04  126.36
2    2023-01-05  125.02
3    2023-01-06  129.62
4    2023-01-09  130.15
..          ...     ...
245  2023-12-22  193.60
246  2023-12-26  193.05
247  2023-12-27  193.15
248  2023-12-28  193.58
249  2023-12-29  192.53

[250 rows x 2 columns]
